# CNN

In [1]:
import time
import ast
import numpy as np
import pandas as pd
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    log_loss
)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

import os, tensorflow as tf
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel("ERROR")

from evaluation import *
from data_utils import *

2025-11-02 23:17:47.522599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-02 23:17:47.522669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-02 23:17:47.524434: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-02 23:17:47.533862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.layers import (
    TextVectorization, Embedding, Conv1D, GlobalMaxPooling1D,
    Dense, Dropout, Input, MaxPooling1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import tensorflow as tf

In [3]:
# Config
try:
    CFG
except NameError:
    class CFG:
        seeds = [42, 119, 2020, 2024, 2028]
        
train_df, test_df, y, class_names = load_and_prepare_data()
pairs_train, pairs_val, test_pairs, y_train, y_val = prepare_dual_conversation_pipeline(train_df, test_df, y)

print("Conv A Example:\n", pairs_train[0][0][:200])
print("Conv B Example:\n", pairs_train[0][1][:200])
print("Label:", y_train[0])

Conv A Example:
 What is a foreign exchange crisis? What are some notable examples?
A foreign exchange crisis refers to a situation where a country faces severe shortage of foreign currencies, usually dollars or euros
Conv B Example:
 What is a foreign exchange crisis? What are some notable examples?
A foreign exchange crisis, also known as a currency crisis or balance of payments crisis, occurs when a country's currency experience
Label: 0


In [4]:
# Vectorizer (shared across the two inputs)
from tensorflow.keras.layers import TextVectorization

vocab_size = 20000
max_length = 512

adapt_strings = [p[0] for p in pairs_train] + [p[1] for p in pairs_train]
adapt_ds = tf.data.Dataset.from_tensor_slices([str(t) for t in adapt_strings]).batch(1024)

text_vectorizer = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=max_length
)
text_vectorizer.adapt(adapt_ds)

2025-11-02 23:18:03.283836: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 626 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0


In [5]:
# tf.data pipelines for ((A,B), y)
def make_dual_dataset(pairs, labels=None, batch_size=128, training=True):
    part_a = [str(p[0]) for p in pairs]
    part_b = [str(p[1]) for p in pairs]

    inputs = {"inp_a": tf.constant(part_a), "inp_b": tf.constant(part_b)}

    if labels is None:
        ds = tf.data.Dataset.from_tensor_slices(inputs)
    else:
        labels = np.asarray(labels, dtype=np.int32)
        ds = tf.data.Dataset.from_tensor_slices((inputs, labels))

    if labels is not None and training:
        ds = ds.shuffle(2048, reshuffle_each_iteration=True)

    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = make_dual_dataset(pairs_train, y_train, training=True)
val_ds   = make_dual_dataset(pairs_val,   y_val,   training=False)
test_ds  = make_dual_dataset(test_pairs,  labels=None,  training=False)

In [6]:
# Model (two string inputs → vectorizer → shared embedding → concat → conv)
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, MaxPooling1D, Concatenate
from tensorflow.keras.models import Model

def get_dual_cnn_model(vocab_size=vocab_size, embed_dim=64, num_classes=3):
    inp_a = Input(shape=(), dtype=tf.string, name="inp_a")
    inp_b = Input(shape=(), dtype=tf.string, name="inp_b")

    # shared layers
    emb   = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)

    # branch A
    xa = text_vectorizer(inp_a)
    xa = emb(xa)
    xa = Conv1D(32, 3, activation="relu")(xa)
    xa = Conv1D(32, 3, activation="relu")(xa)
    xa = MaxPooling1D()(xa)
    xa = Conv1D(64, 3, activation="relu")(xa)
    xa = Conv1D(64, 3, activation="relu")(xa)
    xa = MaxPooling1D()(xa)
    xa = Conv1D(128, 3, activation="relu")(xa)
    xa = Conv1D(128, 3, activation="relu")(xa)
    xa = GlobalMaxPooling1D()(xa)

    # branch B
    xb = text_vectorizer(inp_b)
    xb = emb(xb)
    xb = Conv1D(32, 3, activation="relu")(xb)
    xb = Conv1D(32, 3, activation="relu")(xb)
    xb = MaxPooling1D()(xb)
    xb = Conv1D(64, 3, activation="relu")(xb)
    xb = Conv1D(64, 3, activation="relu")(xb)
    xb = MaxPooling1D()(xb)
    xb = Conv1D(128, 3, activation="relu")(xb)
    xb = Conv1D(128, 3, activation="relu")(xb)
    xb = GlobalMaxPooling1D()(xb)

    # merge
    x  = Concatenate()([xa, xb])
    x  = Dropout(0.3)(x)
    x  = Dense(256, activation="swish")(x)
    out = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=[inp_a, inp_b], outputs=out)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [7]:
# Train one model or an ensemble over seeds
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.makedirs("models_cnn_dual3", exist_ok=True)
cnn_models = []

for seed in CFG.seeds:
    tf.keras.utils.set_random_seed(seed)

    model_path = os.path.join("models_cnn_dual3", f"cnn_dual_seed_{seed}.keras")

    if os.path.exists(model_path):
        print(f"✅ Found existing model for seed {seed}, loading...")
        model = tf.keras.models.load_model(model_path)
    else:
        print(f"🚀 Training new model for seed {seed}...")
        model = get_dual_cnn_model(vocab_size=vocab_size, embed_dim=64, num_classes=3)

        ckpt = ModelCheckpoint(
            filepath=model_path,
            monitor="val_loss",
            mode="min",
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )

        es = EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
        )

        # --- Measure training time ---
        t0 = time.time()
        
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=30,
            callbacks=[ckpt, es],
            verbose=1
        )
        
        t1 = time.time()
        print(f"⏱️ CNN Training time (seed {seed}): {(t1 - t0):.2f} seconds ({(t1 - t0)/60:.2f} minutes)")
        
        # --- Reload best model (for consistency) ---
        model = tf.keras.models.load_model(model_path)

    # --- Evaluate on validation set ---
    loss, acc = model.evaluate(val_ds, verbose=0)
    print(f"[Seed {seed}] Val Loss: {loss:.4f} | Val Acc: {acc*100:.2f}%\n")

    # --- Store model for ensemble ---
    cnn_models.append(model)

🚀 Training new model for seed 42...
Epoch 1/30


2025-11-02 23:18:11.053422: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-11-02 23:18:11.524998: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-11-02 23:18:11.612228: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-11-02 23:18:12.765254: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ffcf42e8210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-02 23:18:12.765291: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2025-11-02 23:18:12.772240: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1762147092.853892 2799103 device_compiler.h:

360/360 [==============================] - ETA: 0s - loss: 1.0937 - accuracy: 0.3745
Epoch 1: val_loss improved from inf to 1.08479, saving model to models_cnn_dual3/cnn_dual_seed_42.keras
360/360 [==============================] - 44s 107ms/step - loss: 1.0937 - accuracy: 0.3745 - val_loss: 1.0848 - val_accuracy: 0.4035
Epoch 2/30
360/360 [==============================] - ETA: 0s - loss: 1.0569 - accuracy: 0.4464
Epoch 2: val_loss did not improve from 1.08479
360/360 [==============================] - 27s 74ms/step - loss: 1.0569 - accuracy: 0.4464 - val_loss: 1.1020 - val_accuracy: 0.4122
Epoch 3/30
360/360 [==============================] - ETA: 0s - loss: 0.9335 - accuracy: 0.5522
Epoch 3: val_loss did not improve from 1.08479
360/360 [==============================] - 22s 61ms/step - loss: 0.9335 - accuracy: 0.5522 - val_loss: 1.2343 - val_accuracy: 0.4006
Epoch 4/30
360/360 [==============================] - ETA: 0s - loss: 0.7716 - accuracy: 0.6449
Epoch 4: val_loss did not imp

In [8]:
print(f"Number of models in cnn_models: {len(cnn_models)}")
if len(cnn_models) > 0:
    for i, m in enumerate(cnn_models):
        preds = m.predict(val_ds, verbose=0)
        print(f"Model {i} predictions shape:", preds.shape)

Number of models in cnn_models: 5
Model 0 predictions shape: (11496, 3)
Model 1 predictions shape: (11496, 3)
Model 2 predictions shape: (11496, 3)
Model 3 predictions shape: (11496, 3)
Model 4 predictions shape: (11496, 3)


In [9]:
# Predictions (CNN)
y_proba_val_cnn = np.mean([m.predict(val_ds,  verbose=0) for m in cnn_models], axis=0)
y_pred_val_cnn  = y_proba_val_cnn.argmax(axis=1)

# Test predictions + submission
test_ds = make_dual_dataset(test_pairs, labels=None, training=False)

y_proba_test_cnn = np.mean([m.predict(test_ds, verbose=0) for m in cnn_models], axis=0)
y_pred_test_cnn  = y_proba_test_cnn.argmax(axis=1)

In [10]:
print("\n================ CNN3 EVALUATION ================\n")
# Metrics
_ = eval_metrics(y_val, y_pred_val_cnn)
eval_classification_report(y_val, y_pred_val_cnn, class_names)
# ROC-AUC
_ = eval_roc_auc(y_val, y_proba_val_cnn)
# Log-loss
_ = eval_log_loss(y_val, y_proba_val_cnn)
_ = eval_log_loss_per_class(y_val, y_proba_val_cnn)


================ CNN3 EVALUATION ================

*** GLOBAL METRICS ***
Accuracy (Global)      : 0.4228
Precision (Macro Avg)  : 0.4317
Recall (Macro Avg)     : 0.4139
F1-Score (Macro Avg)   : 0.3931

*** PER-CLASS EVALUATION ***
Class                Precision    Recall  F1-Score   Support
------------------------------------------------------------
winner_model_a            0.43      0.50      0.46      4013
winner_model_b            0.41      0.58      0.48      3931
winner_tie                0.46      0.16      0.24      3552
------------------------------------------------------------
Macro Avg                 0.43      0.41      0.39     34488
Weighted Avg              0.43      0.42      0.40     34488

*** ROC-AUC EVALUATION ***
ROC-AUC (OvR) : 0.5936

*** LOG-LOSS EVALUATION ***
Log-loss      : 1.0835

*** LOG-LOSS PER CLASS ***
Class 0: 1.0429  (n=4013)
Class 1: 1.0379  (n=3931)
Class 2: 1.1800  (n=3552)


In [11]:
# Confusion Matrix + Plot
cm_cnn = eval_confusion_matrix(y_val, y_pred_val_cnn, n_classes=y_proba_val_cnn.shape[1])
plot_confusion_matrix(cm_cnn, class_names, title="Confusion Matrix — CNN3", save_path="results/confusion_matrix/confusion_matrix_cnn3.png")


Confusion Matrix (rows=true, cols=pred):
 [[2000 1664  349]
 [1311 2294  326]
 [1391 1594  567]]
Saved plot to: images/confusion_matrix/confusion_matrix_cnn3.png


In [12]:
# ROC Curves
plot_roc_curves(y_val, y_proba_val_cnn, class_names, title_prefix="CNN3 ROC", save_path="results/roc/roc_cnn3.png")

Saved plot to: images/roc/roc_cnn3.png


In [13]:
save_roc_to_csv(y_val, y_proba_val_cnn, "CNN3", fold_idx=1)

Saved ROC data for class 0 (AUC=0.6069) → results/roc/CNN3_fold1_class0.csv
Saved ROC data for class 1 (AUC=0.6085) → results/roc/CNN3_fold1_class1.csv
Saved ROC data for class 2 (AUC=0.5652) → results/roc/CNN3_fold1_class2.csv


In [14]:
submission_cnn = build_submission(
    test_df=test_df,
    y_pred_test=y_pred_test_cnn,
    y_proba_test=y_proba_test_cnn,
    model_name="cnn3"
)


Saved: results/submission/submission_cnn3.csv
